In [8]:
from dotenv import load_dotenv

load_dotenv()

True

In [9]:
from typing_extensions import TypedDict
from typing import Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [10]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama-3.3-70b-versatile")

In [11]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state: State) -> State:
    return {
        "messages": [llm.invoke(state["messages"])],
    }

builder = StateGraph(State)
builder.add_node("chatbot_node", chatbot)

builder.add_edge(START, "chatbot_node")
builder.add_edge("chatbot_node", END)

graph = builder.compile()

In [12]:
message = {"role": "user", "content": "Who walked on the moon for the first time? Print only the name"}
response = graph.invoke({"messages": [message]})

response["messages"]

[HumanMessage(content='Who walked on the moon for the first time? Print only the name', additional_kwargs={}, response_metadata={}, id='7feca848-c5e1-4f8f-840c-fd4f53436b38'),
 AIMessage(content='Neil Armstrong', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 49, 'total_tokens': 52, 'completion_time': 0.007968627, 'completion_tokens_details': None, 'prompt_time': 0.001521102, 'prompt_tokens_details': None, 'queue_time': 0.057916878, 'total_time': 0.009489729}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_c06d5113ec', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--0c2403a7-ecca-41cb-8732-8ec8c6c6d5d0-0', usage_metadata={'input_tokens': 49, 'output_tokens': 3, 'total_tokens': 52})]

In [13]:
state = None

while True:
    in_message = input("You: ")

    if in_message.lower() in {"quit", "exit"}:
        break
    if state is None:
        state: State = {
            "messages": [{"role": "user", "content": in_message}],
        }
    else:
        state["messages"].append({"role": "user", "content": in_message})

    state = graph.invoke(state)
    print("Bot: ", state["messages"][-1].content)

Bot:  The capital of India is New Delhi.
Bot:  New Delhi is located in the northern part of India, and it is part of the National Capital Territory of Delhi (NCT). It is situated on the west bank of the Yamuna River and is bordered by the states of Haryana to the north, west, and south, and Uttar Pradesh to the east.


KeyboardInterrupt: Interrupted by user